In [1]:
import pandas as pd
import numpy as np
import os
import folium
from sklearn.preprocessing import OneHotEncoder

In [2]:
FOLDER_PATH = "data"
def print_sources():
    onlyfiles = [f for f in os.listdir(FOLDER_PATH) if os.path.isfile(os.path.join(FOLDER_PATH, f))]
    return onlyfiles

In [3]:
def load_csv(path):
    return pd.read_csv(os.path.join(FOLDER_PATH, path))

In [4]:
def oneHotEncode(df, cats):
    #One hot encoder
    cat_encoder = OneHotEncoder()
    party_cat = df[cats]
    one_hot_party = cat_encoder.fit_transform(party_cat)
    one_hot_party = one_hot_party.toarray()
    return one_hot_party

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
def minMaxScale(df, cols):
    scaler = MinMaxScaler()
    return scaler.fit_transform(df[cols])

In [6]:
print_sources()

['archive.zip',
 'governors_county.csv',
 'governors_county_candidate.csv',
 'governors_state.csv',
 'house_candidate.csv',
 'house_state.csv',
 'pres16results.csv',
 'president_county.csv',
 'president_county_candidate.csv',
 'president_state.csv',
 'senate_county.csv',
 'senate_county_candidate.csv',
 'senate_state.csv']

In [7]:
house_data = load_csv('house_state.csv')
president_data = load_csv('president_county_candidate.csv')
senate_data = load_csv('senate_county_candidate.csv')

In [8]:
def drop_dups_and_encode(df):
    df['county_state'] = df['county'] +'_'+ df['state']
    votes = df.groupby(['county_state','party'], as_index=False).sum()
    print(type(votes))
    votes = votes.sort_values(['county_state','party']).reset_index(drop=True)
    df = df.drop_duplicates(['county_state','party'])
    df = df.sort_values(['county_state','party']).reset_index(drop=True)
    df[['party_votes']] = votes['total_votes']
    #print(df.head())
    return df

In [9]:
def filter_parties(df):
    third_f = (df['party'] != 'DEM') & (df['party'] != 'REP')
    df.loc[third_f,'party'] = '3RD'
    df = df.groupby(['party','state','county'])['total_votes'].sum().reset_index()
    #df = df.drop(['total_votes'],axis=1)
    return df

In [10]:
def sumGroupColumn(df,groups,column):
    return df.groupby(groups,as_index=False)[column].sum()[column]
def sum_col_compact(df,groups,column):
    df2 = df.groupby(groups).agg({column:np.sum}).reset_index()
    return df2

In [11]:
pres_data_r = filter_parties(president_data)
pres_data_r = sum_col_compact(pres_data_r,['state','party'],'total_votes')
state_votes = president_data.groupby('state')['total_votes'].sum().reset_index()
pres_data_r = pres_data_r.rename(columns={'total_votes': 'party_votes'})
state_votes = state_votes.rename(columns={'total_votes': 'state_votes'})
pres_data_r = pres_data_r.merge(state_votes, on='state')
pres_data_r.head()

,state,party,party_votes,state_votes
0,Alabama,3RD,32268,2309900
1,Alabama,DEM,843473,2309900
2,Alabama,REP,1434159,2309900
3,Alaska,3RD,13442,349473
4,Alaska,DEM,150262,349473


In [12]:
pres_data_r['pct'] = pres_data_r['party_votes']/pres_data_r['state_votes']
pres_data_r.loc[:,'pct_mm'] = minMaxScale(pres_data_r,['pct'])
pres_data_3rd = pres_data_r[pres_data_r['party'] == '3RD']
pres_data_3rd.sort_values('pct',ascending=False)
pres_data_3rd.head()

,state,party,party_votes,state_votes,pct,pct_mm
0,Alabama,3RD,32268,2309900,0.013969,0.005266
3,Alaska,3RD,13442,349473,0.038464,0.032133
6,Arizona,3RD,52016,3385845,0.015363,0.006794
9,Arkansas,3RD,34289,1212065,0.028290,0.020974
12,California,3RD,362794,16909159,0.021455,0.013477


In [13]:
pct_3p_map = folium.Map(location=[48, -102], zoom_start=3)
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
state_geo = f'{url}/us-states.json'
folium.Choropleth(geo_data=state_geo, data=pres_data_3rd,
             columns=['state', 'pct_mm'],
             key_on='feature.properties.name',
             fill_color='YlGn', fill_opacity=0.7, line_opacity=0.2,
             legend_name='Participation Rate (%)').add_to(pct_3p_map)
pct_3p_map

Let's check out 3rd party votes in the 2016 election

In [14]:
pres_16_df = load_csv('pres16results.csv')
pres_16_df.head()

,county,fips,cand,st,pct_report,votes,total_votes,pct,lead
0,NaN,US,Donald Trump,US,0.9951,60350241.0,127592176.0,0.472993,Donald Trump
1,NaN,US,Hillary Clinton,US,0.9951,60981118.0,127592176.0,0.477938,Donald Trump
2,NaN,US,Gary Johnson,US,0.9951,4164589.0,127592176.0,0.032640,Donald Trump
3,NaN,US,Jill Stein,US,0.9951,1255968.0,127592176.0,0.009844,Donald Trump
4,NaN,US,Evan McMullin,US,0.9951,451636.0,127592176.0,0.003540,Donald Trump


In [15]:
pres_16_df = pres_16_df[pres_16_df['st']  != 'US']
pres_16_df = pres_16_df.dropna(0,subset=['st'])
pres_16_df['st'].unique().size #confirm we have 51 states

51

In [16]:
states_converter = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}
states_converter_inv = {v: k for k, v in states_converter.items()}



In [17]:
pres_16_df['state'] = pres_16_df['st'].apply(lambda x: states_converter_inv[x])
pres_16_df.head()

,county,fips,cand,st,pct_report,votes,total_votes,pct,lead,state
32,NaN,CA,Hillary Clinton,CA,1.0,5931283.0,9631972.0,0.615791,Hillary Clinton,California
33,NaN,CA,Donald Trump,CA,1.0,3184721.0,9631972.0,0.330641,Hillary Clinton,California
34,NaN,CA,Gary Johnson,CA,1.0,308392.0,9631972.0,0.032018,Hillary Clinton,California
35,NaN,CA,Jill Stein,CA,1.0,166311.0,9631972.0,0.017267,Hillary Clinton,California
36,NaN,CA,Gloria La Riva,CA,1.0,41265.0,9631972.0,0.004284,Hillary Clinton,California


In [18]:
third_locations = (pres_16_df['cand'] != 'Donald Trump') & (pres_16_df['cand'] != 'Hillary Clinton')
pres_16_df.loc[third_locations,'cand'] = '3RD'
pres16_state = pres_16_df.groupby(['state','cand']).aggregate({'votes':sum,'total_votes':sum}).reset_index()
pres16_state = pres16_state.rename(columns={"cand":'party'})
pres16_state.loc[pres16_state['party'] == 'Donald Trump','party'] = 'REP'
pres16_state.loc[pres16_state['party'] == 'Hillary Clinton','party'] = 'DEM'

In [31]:
pres16_state['pct'] = pres16_state['votes']/pres16_state['total_votes']
pres16_state.loc[:,'pct_mm'] = minMaxScale(pres16_state,['pct'])

In [32]:
pct_3p_map16 = folium.Map(location=[48, -102], zoom_start=3)
pres16_state_3rd = pres16_state.loc[pres16_state['party']=='3RD']
folium.Choropleth(geo_data=state_geo, data=pres16_state_3rd,
             columns=['state', 'pct'],
             key_on='feature.properties.name',
             fill_color='YlGn', fill_opacity=0.7, line_opacity=0.2,
             legend_name='Participation Rate (%)').add_to(pct_3p_map16)
pct_3p_map16

In [33]:
print(pres_data_r.head())
print(pres16_state)

     state party  party_votes  state_votes       pct    pct_mm
0  Alabama   3RD        32268      2309900  0.013969  0.005266
1  Alabama   DEM       843473      2309900  0.365156  0.390481
2  Alabama   REP      1434159      2309900  0.620875  0.670978
3   Alaska   3RD        13442       349473  0.038464  0.032133
4   Alaska   DEM       150262       349473  0.429967  0.461573
         state party      votes  total_votes       pct    pct_mm
0      Alabama   3RD   106312.0    8312660.0  0.012789  0.010420
1      Alabama   REP  2613850.0    4156330.0  0.628884  0.676239
2      Alabama   DEM  1436168.0    4156330.0  0.345538  0.370024
3       Alaska   3RD    46332.0    1972704.0  0.023487  0.021981
4       Alaska   REP   260830.0     493176.0  0.528878  0.568162
..         ...   ...        ...          ...       ...       ...
148  Wisconsin   REP  2818934.0    5889240.0  0.478658  0.513889
149  Wisconsin   DEM  2764420.0    5889240.0  0.469402  0.503886
150    Wyoming   3RD    37090.0    19

In [34]:
compare_pcts_df = pres_data_r.merge(pres16_state,on=['state','party'])
compare_pcts_df = compare_pcts_df.rename(columns={'pct_x' : 'pct_20', 'pct_y' : 'pct_16','pct_mm_x': 'pctmm_20','pct_mm_y':'pctmm_16'})

In [35]:
compare_pcts_3rd = compare_pcts_df[compare_pcts_df['party']=='3RD']
compare_pcts_3rd.head()
compare_pcts_3rd.loc[:,['pctmm_20','pctmm_16']] = minMaxScale(compare_pcts_3rd,['pct_20','pct_16'])
compare_pcts_3rd.head()

C:\Users\JBKec\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,state,party,party_votes,state_votes,pct_20,pctmm_20,votes,total_votes,pct_16,pctmm_16
0,Alabama,3RD,32268,2309900,0.013969,0.147742,106312.0,8312660.0,0.012789,0.177629
3,Alaska,3RD,13442,349473,0.038464,0.901552,46332.0,1972704.0,0.023487,0.374696
6,Arizona,3RD,52016,3385845,0.015363,0.190623,210812.0,8251240.0,0.025549,0.412693
9,Arkansas,3RD,34289,1212065,0.028290,0.588450,130102.0,13460208.0,0.009666,0.120089
12,California,3RD,362794,16909159,0.021455,0.378125,1031936.0,57791832.0,0.017856,0.270972


In [62]:
pct_3p_filter = compare_pcts_3rd.drop(['pct_20','pct_16','votes','party','state_votes','total_votes','party_votes'], axis=1)
pct_3p_filter.head()
pct_3p_filter['diff']= pct_3p_filter['pctmm_20']- pct_3p_filter['pctmm_16'] 
pct_3p_filter['index'] = pct_3p_filter.index

In [63]:
import branca.colormap as cmp
linear = cmp.LinearColormap(
    ['red', 'white', 'blue'],
    vmin=-1, vmax=1,
    caption='Color Scale for Map' #Caption for Color scale or Legend
)
linear

In [64]:
pct_3p_diff_map = folium.Map(location=[48, -102], zoom_start=3)
diff_dict = pct_3p_filter.set_index('state')['diff']
folium.Choropleth(geo_data=state_geo, data=pct_3p_filter,
                  style_function=lambda feature: {
                    'fillColor': linear(diff_dict[feature['index']]),
                    'color': 'black',       #border color for the color fills
                    'weight': 1,            #how thick the border has to be
                    'dashArray': '5, 3'  #dashed lines length,space between them
                },
                  key_on='feature.properties.name',
                 legend_name='Participation Rate (%)').add_to(pct_3p_diff_map)
pct_3p_diff_map

TypeError: 'NoneType' object is not subscriptable